# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [ ]:
#Produto escolhido:
produto = 'Trivago'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [20]:
xls = pd.ExcelFile('Trivago.xlsx')
Treinamento= xls.parse('Treinamento')
Teste = xls.parse('Teste')

In [21]:
dt = pd.read_excel('Trivago.xlsx')

Ind = ["+","…","(","|","^","'","  ","   ","    ","html","@","#","rt",":",",",";",".","?","!","/",")","\n","  ","    ","     "]
       
contador = 0

for i in dt.Treinamento:
    
    for b in Ind:
        i=i.replace(b," ")
        
    while True:
        if i[-1] == " ":
            i = i[:len(i)-1]
        else:
            break
            
    dt.Treinamento[contador]= i
    contador+=1
    
dt

,Treinamento,Classificação
0,eu devo ser a unica pessoa no mundo q ainda ac...,Relevante
1,soufrasesz amor da sua vida= 13 letras euuuu...,Irrelevante
2,eu não aguento mais esse comercial do trivago pqp,Relevante
3,sasaqui_isabel vendo os clipes antigos da an...,Irrelevante
4,caraca trivago é internacional hsuahsuash ahs7...,Irrelevante
5,soufrasesz amor da sua vida= 13 letras euuuu...,Irrelevante
6,vocenaosabiaq \rquando vc para pra pensar q a...,Irrelevante
7,soufrasesz amor da sua vida= 13 letras euuuu...,Irrelevante
8,undeadybala aí um dos motivos pra usar trivag...,Relevante
9,soufrasesz amor da sua vida= 13 letras euuuu...,Irrelevante


In [22]:
classi = dt.Classificação
teste = dt.Treinamento
dictP = dict()

for leo in range (0, len(teste)):
    situação = classi[leo]
    tweet = teste[leo].split()
    for palavras in tweet:

        if palavras in dictP:
            if situação == "Relevante" or situação == "relevante":
                dictP[palavras][0] += 1
            else:
                dictP[palavras][1] += 1
        else:
            if situação == "Relevante" or situação == "relevante":
                dictP[palavras] = [1,0]
            else:
                dictP[palavras] = [0,1]

                
base_rel = 0 
base_irr = 0
for values in dictP:
    base_rel += dictP[values][0]
    base_irr += dictP[values][1]
    
probabilidade_rel = base_rel/(base_rel+base_irr)
probabilidade_irr = 1 - probabilidade_rel
print("Probabilidade de não ser relevante",probabilidade_irr)
print("Probabilidade de ser relevante",probabilidade_rel)

len_dict = len(dictP)

Probabilidade de não ser relevante 0.9380191693290735
Probabilidade de ser relevante 0.061980830670926516


In [23]:
twit_rel = dt[dt.Classificação == 'Relevante']
relev = np.sum(twit_rel.Treinamento + " ").split()
cr = pd.Series(relev)
total_relevantes= len(relev)
total_relevantes

291

In [24]:
twit_irrel = dt[dt.Classificação == 'Irrelevante']
irrelev = np.sum(twit_irrel.Treinamento + " ").split()
cirr=pd.Series(irrelev)
total_irrelevantes=len(cirr)
total_irrelevantes

4404

Teste

In [28]:
teste = Teste.Teste
classi = Teste.Classificação
dictP_teste = dict()
probabilidade_palavra_rel = dict()
probabilidade_palavra_irrel = dict()

for y in range (0, len(teste)):
    probabilidade_frase_relevante = 1
    probabilidade_frase_irrelevante = 1
    
    for palavra in tweet:
        if palavra in dictP:
            probabilidade_palavra_rel[palavra] = (dictP[palavra][0] + 1)/(total_relevantes + len_dict)
            probabilidade_palavra_irrel[palavra] = (dictP[palavra][1] + 1)/(total_irrelevantes + len_dict)
            
            probabilidade_frase_relevante *= probabilidade_palavra_rel[palavra]
            probabilidade_frase_irrelevante *= probabilidade_palavra_irrel[palavra]
        
        else:
            probabilidade_palavra_rel[palavra] = 1/(total_bom + len_dict)
            probabilidade_palavra_irrel[palavra] = 1/(total_bom + len_dict)
            
            probabilidade_frase_relevante *= probabilidade_palavra_rel[palavra]
            probabilidade_frase_irrelevante *= probabilidade_palavra_irrel[palavra]

    if probabilidade_frase_relevante > probabilidade_frase_irrelevante:

        Teste.loc[(Teste.Teste == teste[y]),'Resultado'] = 'Relevante'
        
    else:
        
        Teste.loc[(Teste.Teste == teste[y]),'Resultado'] = 'Irrelevante'

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [32]:
Teste.loc[(Teste.Classificação == "Irrelevante")& (Teste.Resultado == "Irrelevante"),'Performance'] = 'NV'
Teste.loc[(Teste.Classificação == "Relevante")& (Teste.Resultado == "Relevante"),'Performance'] = 'PV'
Teste.loc[(Teste.Classificação == "Irrelevante")& (Teste.Resultado == "Relevante"),'Performance'] = 'PF'
Teste.loc[(Teste.Classificação == "Relevante")& (Teste.Resultado == "Irrelevante"),'Performance'] = 'NF'


In [36]:
um = Teste.Performance.value_counts()[0]

dois = 0 # não houveram positivos verdadeiros

tres = Teste.Performance.value_counts()[1]

quatro = 0  # não houveram positivos falsos


Porcentagem_um = um/(um+dois+tres+quatro)*100
Porcentagem_dois = dois/(um+dois+tres+quatro)*100
Porcentagem_tres = tres/(um+dois+tres+quatro)*100
Porcentagem_quatro = quatro/(um+dois+tres+quatro)*100

print("\n")

print("porcentagem de negativos verdadeiros {0}%\nporcentagem de positivos verdadeiros {1}%\nporcentagem de negativos falsos {2}%\nporcentagem de positivos falsos:{3} %".format(Porcentagem_um,Porcentagem_dois,Porcentagem_tres,Porcentagem_quatro))



porcentagem de negativos verdadeiros 74.5%
porcentagem de positivos verdadeiros 0.0%
porcentagem de negativos falsos 25.5%
porcentagem de positivos falsos:0.0 %


___
## Concluindo

O nosso programa de machine learning construído a partir da teoria "Naive Bayes’, apresenta uma boa porcentagem de acertos mesmo levando em conta a baixíssima porcentagem de tuites relevantes. A porcentagem de acerto do classificador relacionado a tuites irrelevantes é cerca de 71,5% de precisão, e porcentagem de positivos verdadeiros não se distanciando muito da taxa real, 6,1%. Assim podendo ser considerado o machine learning aceitável.    .<br />

Porcentagem de negativos verdadeiros 74.5%.<br />
Porcentagem de positivos verdadeiros 0.0%.<br />
Porcentagem de negativos falsos 25.5%.<br />
Porcentagem de positivos falsos:0.0%.<br />

As mensagens com dupla negação e sarcasmo são tratadas como negativos verdadeiros e falsos, pois o classificado só considera a probabilidade de palavras e não o seu contexto.

O nosso código já apresenta bons resultados e apenar precisa de um pouco mais de tuites classificados para que a sua precisão fique ainda maior, assim o investimento seria pequeno e apresentaria um retorno rápido.



In [35]:
print("porcentagem de negativos verdadeiros {0}%\nporcentagem de positivos verdadeiros {1}%\nporcentagem de negativos falsos {2}%\nporcentagem de positivos falsos:{3} 0%".format(Porcentagem_um,Porcentagem_dois,Porcentagem_tres,Porcentagem_quatro))

porcentagem de negativos verdadeiros 74.5%
porcentagem de positivos verdadeiros 0.0%
porcentagem de negativos falsos 25.5%
porcentagem de positivos falsos:0.0 0%
